In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
df=pd.read_csv(r'pcs_data (04182022)_VRVSVT.csv')
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.set_index('Timestamp')
df.head()

,acVR,acVS,acVT
Timestamp,,,
2022-04-14 02:16:00,398.0,394.4,396.8
2022-04-14 02:16:00,397.5,394.7,397.0
2022-04-14 02:16:00,397.5,394.7,397.0
2022-04-14 02:16:00,397.5,395.3,398.1
2022-04-14 02:16:00,397.5,395.3,398.1


In [4]:
daily_df = df.copy()
print(len(daily_df))
daily_df.head()


256345


,acVR,acVS,acVT
Timestamp,,,
2022-04-14 02:16:00,398.0,394.4,396.8
2022-04-14 02:16:00,397.5,394.7,397.0
2022-04-14 02:16:00,397.5,394.7,397.0
2022-04-14 02:16:00,397.5,395.3,398.1
2022-04-14 02:16:00,397.5,395.3,398.1


In [5]:
train_df,test_df = daily_df[:100000], daily_df[100000:120000] 


In [6]:
print((train_df).shape)

(100000, 3)


In [7]:
train = train_df
scalers={}
for i in train_df.columns:
    scaler = MinMaxScaler()
    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    train[i]=s_s
test = test_df
for i in train_df.columns:
    scaler = scalers['scaler_'+i]
    s_s = scaler.transform(test[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+i] = scaler
    test[i]=s_s

<ipython-input-7-538f0eb9a691>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i]=s_s
<ipython-input-7-538f0eb9a691>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i]=s_s
<ipython-input-7-538f0eb9a691>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr

In [8]:
def split_series(series, n_past, n_future):
    X, y = list(), list()
    for window_start in range(len(series)):
        past_end = window_start + n_past
        future_end = past_end + n_future
        if future_end > len(series):
            break
        # slicing the past and future parts of the window
        past, future = series[window_start:past_end, :], series[past_end:future_end, :]
        X.append(past)
        y.append(future)
    return np.array(X), np.array(y)

In [9]:
n_past = 120
n_future = 12 
n_features = 3

In [10]:
X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

In [11]:
print(X_train)

[[[0.45394737 0.39215686 0.45512821]
  [0.42105263 0.41176471 0.46794872]
  [0.42105263 0.41176471 0.46794872]
  ...
  [0.40789474 0.39215686 0.46153846]
  [0.40789474 0.39215686 0.46153846]
  [0.40131579 0.41830065 0.41025641]]

 [[0.42105263 0.41176471 0.46794872]
  [0.42105263 0.41176471 0.46794872]
  [0.42105263 0.45098039 0.53846154]
  ...
  [0.40789474 0.39215686 0.46153846]
  [0.40131579 0.41830065 0.41025641]
  [0.40131579 0.41830065 0.41025641]]

 [[0.42105263 0.41176471 0.46794872]
  [0.42105263 0.45098039 0.53846154]
  [0.42105263 0.45098039 0.53846154]
  ...
  [0.40131579 0.41830065 0.41025641]
  [0.40131579 0.41830065 0.41025641]
  [0.40789474 0.41176471 0.49358974]]

 ...

 [[0.82894737 0.78431373 0.83974359]
  [0.82894737 0.78431373 0.83974359]
  [0.73684211 0.84313725 0.81410256]
  ...
  [0.80921053 0.80392157 0.86538462]
  [0.79605263 0.80392157 0.86538462]
  [0.79605263 0.80392157 0.86538462]]

 [[0.82894737 0.78431373 0.83974359]
  [0.73684211 0.84313725 0.81410256]


In [12]:
print(X_train.shape,y_train.shape)

(99869, 120, 3) (99869, 12, 3)


In [13]:
# E1D1
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)

encoder_states1 = encoder_outputs1[1:]

#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])

#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)

#
model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

#
model_e1d1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 120, 3)]     0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 100), (None, 41600       input_1[0][0]                    
__________________________________________________________________________________________________
repeat_vector (RepeatVector)    (None, 12, 100)      0           lstm[0][0]                       
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 12, 100)      80400       repeat_vector[0][0]              
                                                                 lstm[0][1]                   

In [14]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)


In [15]:
model_e1d1.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e1d1=model_e1d1.fit(X_train,y_train,epochs=50,validation_data=(X_test,y_test),batch_size=1024,verbose=1,callbacks=[reduce_lr])


Epoch 1/50
98/98 [==============================] - 11s 36ms/step - loss: 0.0212 - val_loss: 9.7374e-04
Epoch 2/50
98/98 [==============================] - 2s 22ms/step - loss: 9.5693e-04 - val_loss: 8.9475e-04
Epoch 3/50
98/98 [==============================] - 2s 21ms/step - loss: 9.3854e-04 - val_loss: 8.6524e-04
Epoch 4/50
98/98 [==============================] - 2s 22ms/step - loss: 9.2419e-04 - val_loss: 8.3092e-04
Epoch 5/50
98/98 [==============================] - 2s 22ms/step - loss: 9.0532e-04 - val_loss: 8.1529e-04
Epoch 6/50
98/98 [==============================] - 2s 23ms/step - loss: 8.9061e-04 - val_loss: 7.8547e-04
Epoch 7/50
98/98 [==============================] - 2s 23ms/step - loss: 8.7629e-04 - val_loss: 7.7542e-04
Epoch 8/50
98/98 [==============================] - 2s 21ms/step - loss: 8.6821e-04 - val_loss: 8.1061e-04
Epoch 9/50
98/98 [==============================] - 2s 22ms/step - loss: 8.6740e-04 - val_loss: 7.5740e-04
Epoch 10/50
98/98 [=====================

In [16]:
pred_e1d1=model_e1d1.predict(X_test)
pred_e1d=model_e1d1.predict(X_train)


In [17]:
for index,i in enumerate(train_df.columns):
    scaler = scalers['scaler_'+i]
    #pred1_e1d1[:,:,index]=scaler.inverse_transform(pred1_e1d1[:,:,index])
    pred_e1d1[:,:,index]=scaler.inverse_transform(pred_e1d1[:,:,index])
    pred_e1d[:,:,index]=scaler.inverse_transform(pred_e1d[:,:,index])
    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])

In [18]:
from sklearn.metrics import mean_absolute_error
for index,i in enumerate(train_df.columns):
    print(i)
    for j in range(1,12):
        print("Second ",j,":")
        print("MAE-E1D1 : ",mean_absolute_error(y_test[:,j-1,index],pred_e1d1[:,j-1,index]),end=", ")
        print()
        print()

acVR
Second  1 :
MAE-E1D1 :  0.3996989036735542, 

Second  2 :
MAE-E1D1 :  0.41299330930092143, 

Second  3 :
MAE-E1D1 :  0.4159441516002083, 

Second  4 :
MAE-E1D1 :  0.42088622647530816, 

Second  5 :
MAE-E1D1 :  0.4299292169127712, 

Second  6 :
MAE-E1D1 :  0.43807529282621516, 

Second  7 :
MAE-E1D1 :  0.44412330267647654, 

Second  8 :
MAE-E1D1 :  0.44793230191445405, 

Second  9 :
MAE-E1D1 :  0.4499826681536528, 

Second  10 :
MAE-E1D1 :  0.4565471677008824, 

Second  11 :
MAE-E1D1 :  0.46819153461766383, 

acVS
Second  1 :
MAE-E1D1 :  0.3907762528521777, 

Second  2 :
MAE-E1D1 :  0.40842046010499616, 

Second  3 :
MAE-E1D1 :  0.4141528273619942, 

Second  4 :
MAE-E1D1 :  0.4210982625896888, 

Second  5 :
MAE-E1D1 :  0.42794995544792974, 

Second  6 :
MAE-E1D1 :  0.4346104647182205, 

Second  7 :
MAE-E1D1 :  0.44135510752164936, 

Second  8 :
MAE-E1D1 :  0.44586893609932154, 

Second  9 :
MAE-E1D1 :  0.4479770920550109, 

Second  10 :
MAE-E1D1 :  0.4545665450866258, 

Second  11 

In [19]:
model_e1d1.save('mmLSTM_VRVSVT.h5')

In [22]:
print(pred_e1d1[0])

[[402.9827  400.40732 402.12393]
 [402.97043 400.37894 402.0256 ]
 [402.97128 400.36923 401.98914]
 [402.9784  400.36343 401.97104]
 [402.98376 400.36615 401.96194]
 [402.98413 400.37326 401.95612]
 [402.97974 400.3788  401.94977]
 [402.96866 400.38077 401.94064]
 [402.959   400.37732 401.93176]
 [402.9462  400.36865 401.91986]
 [402.9342  400.3541  401.90552]
 [402.92032 400.33685 401.8907 ]]


In [23]:
print(y_test[0])

[[402.7 400.5 402.5]
 [403.8 400.9 402.2]
 [403.8 400.9 402.2]
 [403.  401.3 402.5]
 [403.  401.3 402.5]
 [404.1 400.8 403.1]
 [404.1 400.8 403.1]
 [403.4 401.3 402.4]
 [403.4 401.3 402.4]
 [402.9 400.6 402.9]
 [402.9 400.6 402.9]
 [404.4 400.9 403.2]]
